## Instructions

1) Mettre les fichiers `ratings.csv` et `links.csv` dans le dossier `src/`

2) Renommer les fichiers en `Movielens_ratings.csv` et `Movielens_links.csv`

3) Lancer le notebook

4) Le fichier `Movielens_ratings_updated.csv` va se générer dans le dossier `src/`

## Packages

In [2]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

## Weighted rating IMDB

In [3]:
# Import des fichiers CSV movielens_links et TMDB_movie_vote
links = pd.read_csv('src/Movielens_links.csv')
movie = pd.read_csv('src/TMDB_movie_vote.csv')

# Merge en un seul dataframe movie
movie = pd.merge(movie, links, left_on='id', right_on='tmdbId', how='left')
movie = movie.loc[:,['id', 'movieId', 'vote_count', 'vote_average']]
movie.rename(columns={'id':'tmdb_id', 'movieId' : 'movielens_id'}, inplace=True)

# Application du weighted rating IMDB
vote_counts = movie[movie['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = movie[movie['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.8)
qualified = movie[(movie['vote_count'] >= m) & (movie['vote_count'].notnull()) & (movie['vote_average'].notnull())]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')

def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['score_imdb'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('score_imdb', ascending=False).head(qualified.shape[0])

print(f'Vote average : {C}')
print(f'Nombre de votes minimum : {m}')
print(f'Nombre de films : {qualified.shape[0]}')

Vote average : 5.85045981607357
Nombre de votes minimum : 1707.0
Nombre de films : 2002


## Create TMDB weighted Movie CSV with IMDB weighted CSV

In [5]:
# Filtre des colonnes
weighted_movies = qualified.loc[:,['tmdb_id', 'score_imdb']]

# Export du fichier TMDB_weighted_movies.csv
weighted_movies.to_csv('src/TMDB_weighted_movies.csv', index=False)

## Update Movielens ratings CSV

In [25]:
# Importation du fichier Movielens_ratings.csv
ratings = pd.read_csv('src/Movielens_ratings.csv')
ratings = pd.merge(qualified, ratings, left_on='movielens_id', right_on='movieId', how='left')
ratings = ratings.loc[:, ['tmdb_id', 'userId', 'rating']]
ratings = ratings[ratings['userId'].notnull()]

# Obtenir le compte des votes par utilisateur
user_counts = ratings['userId'].value_counts()

# Filtrer les utilisateurs ayant au moins 20 votes mais moins de 84 votes
borne_inf = 20
borne_sup = 130

filtered_users = user_counts[(user_counts >= borne_inf) & (user_counts < borne_sup)].index

# Appliquer le filtrage sur le DataFrame original
filter_df = ratings[ratings['userId'].isin(filtered_users)]

# Sample d'utilisateurs en fonction du nombre lignes souhaitées
mediane = filter_df['userId'].value_counts().quantile(0.5)
nombre_lignes = 300000
nombre_users = int(nombre_lignes/mediane)
users_keep = filter_df['userId'].value_counts().sample(nombre_users, random_state = 0).index
filter_df = filter_df[filter_df['userId'].isin(users_keep)]

# Create a mapping dictionary
user_mapping = {old_id: new_id for new_id, old_id in enumerate(filter_df['userId'].unique(), start=1)}

# Update the userId column using the mapping
filter_df['userId'] = filter_df['userId'].map(user_mapping)

# Export Movielens_ratings_updated.csv
filter_df.to_csv('src/Movielens_ratings_updated.csv', index=False)

# Suppression du fichier Movielens_ratings.csv
current_directory = os.getcwd()
relative_path = 'src\\Movielens_ratings.csv'
full_path = os.path.join(current_directory, relative_path)

# Vérifier si le fichier existe avant de le supprimer
if os.path.exists(full_path):
    os.remove(full_path)
    print(f"Le fichier {relative_path} a été supprimé avec succès.")
else:
    print(f"Le fichier {relative_path} n'existe pas dans le dossier spécifié.")